# Выбор локации для скважины

Для добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Импорт библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statistics 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from scipy import stats as st
from sklearn.metrics import confusion_matrix


Константы:

In [2]:
RANDOM_ST = 12345
PRICE = 450 #цена за 1000 баррелей
COUNT = 200 #количество скважин
COST = 10000000000 #Стоимость разработки 200 скважин

Скытие предупреждений:

In [3]:
pd.options.mode.chained_assignment = None

Загрузка данных геологоразведки трёх регионов:

In [4]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


f0, f1, f2 — три признака точек, описание не дано,но сами признаки значимы.

Пропусков нет, уже неплохо

In [7]:
geo_data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
geo_data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [9]:
geo_data_1.duplicated().sum()

0

Проверим столбец с идентификаторами для поиска скрытых дубликатов:

In [10]:
geo_data_1['id'].value_counts()

LHZR0    2
5ltQ6    2
wt4Uk    2
bfPNe    2
7D5aO    1
        ..
VIKiF    1
tQrbZ    1
LL71A    1
ZUnOQ    1
fIWTv    1
Name: id, Length: 99996, dtype: int64

Скрытые дубликаты есть. В описании данных сказано, что идентификатор станции уникальный. Но вот мы видим, что он встречается дважды. Это данные за другой период? Это номер скважины а и скважины б, но без а и б? В реальности нужно запросить пояснения, но у нас такой возможности нет. Менять идентификаторы мы не будем, следовательно, так как идентификаторы уйдут из таблицы и анализ по ним проводится не будет(останутся только индексы), то это будут две отдельные строки.

In [11]:
geo_data_1[geo_data_1['id']== 'bxg6G']

,id,f0,f1,f2,product


In [12]:
geo_data_1[geo_data_1['id']== 'fiKDv']

,id,f0,f1,f2,product


In [13]:
#sns.pairplot(geo_data_0,  hue = 'product')

In [14]:
geo_data_0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [15]:
geo_data_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [16]:
geo_data_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Мы видим среднюю обратную зависимость между показателяими f0 и f1, в таблице корреляции её значение -0.44. Для исключения влияния мультиколлинеарности следовало бы проверить, не нужно ли исключить один из показателей, однако у нас информация, что данные готовы к обработке, оставим как есть.

Выкидываем столбец идентификаторов:

In [17]:
geo = [geo_data_0, geo_data_1, geo_data_2]
for index, df in enumerate(geo):
    geo[index]= df.drop(['id'], axis=1)
    print(geo[index].columns)
print(geo[0].columns)

Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')


Наш датафрейм исправлен внутри списка. На всякий случай и файл сделаем вручную:

In [18]:
geo_data_0 = geo_data_0.drop(['id'], axis=1)
print(geo_data_0.columns)
geo_data_1 = geo_data_1.drop(['id'], axis=1)
print(geo_data_1.columns)
geo_data_2 = geo_data_2.drop(['id'], axis=1)
print(geo_data_2.columns)

Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')


**Вывод** наши данные содержали скрытые дубликаты, сейчас каждая строка будет анализироваться без учета этого, так как удален столбец 'id'. Имеются признаки мультиколлинеарности:  между показателяими f0 и f1 присутствует средняя обратная зависимость, в таблице корреляции её значение -0.44. Мы считаем данные готовыми к моделированию.

## Обучение и проверка модели

## Подготовка выборок

Обучим и проверим модель для каждого региона. 

In [19]:
features = []
for index, df in enumerate(geo):
    features.append(df.drop(['product'], axis=1))

features_0 = geo_data_0.drop(['product'], axis=1)
features_1 = geo_data_1.drop(['product'], axis=1)
features_2 = geo_data_2.drop(['product'], axis=1)

In [20]:
target = []
for index, df in enumerate(geo):
    target.append(df['product'])
    print(target[index].shape)

# Хотела красиво, но на этапе разбивки с appendом никак не работало, пришлось вернуть обычное
target_0 = geo_data_0['product']
target_1 = geo_data_1['product']
target_2 = geo_data_2['product']

(100000,)
(100000,)
(100000,)


Разобьём данные на обучающую и валидационную выборки в соотношении 75:25

In [21]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=RANDOM_ST)#, stratify=target_0)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=RANDOM_ST)#, stratify=target_1)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=RANDOM_ST)#, stratify=target_2)

In [22]:
#target_train = []
#target_valid = []
#features_train = []
#features_valid = []
#targets_valid = []
#for regions in range(len(geo)):
#   data = geo[regions]
#    features = data.drop('product', axis=1)
#    target = data['product']
#    features_train, features_valid, target_train, target_valid = (
#        train_test_split( features, target, test_size=0.25, random_state=RANDOM_ST))
#    targets_valid.append(target_valid.reset_index(drop=True))
#   s_predictions.append(pd.Series(predictions))


In [23]:
features_train_0.shape[0]/(features_train_0.shape[0]+features_valid_0.shape[0])

0.75

In [24]:
features_train_1.shape[0]/(features_train_1.shape[0]+features_valid_1.shape[0])

0.75

In [25]:
features_train_2.shape[0]/(features_train_2.shape[0]+features_valid_2.shape[0])

0.75

Идеальные проценты, метод справился.

In [26]:
target_valid = [target_valid_0, target_valid_1, target_valid_2]
features_train = [features_train_0, features_train_1, features_train_2]
features_valid = [ features_valid_0, features_valid_1, features_valid_2]
target_train = [target_train_0, target_train_1, target_train_2]

In [28]:
for index, df in enumerate(target_valid):
    df = df.reset_index(drop= True)
    print(df.head())


0     10.038645
1    114.551489
2    132.603635
3    169.072125
4    122.325180
Name: product, dtype: float64
0    80.859783
1    53.906522
2    30.132364
3    53.906522
4     0.000000
Name: product, dtype: float64
0     61.212375
1     41.850118
2     57.776581
3    100.053761
4    109.897122
Name: product, dtype: float64


In [29]:
target_valid_0 = target_valid_0.reset_index(drop= True)
target_valid_1 = target_valid_1.reset_index(drop= True)
target_valid_2 = target_valid_2.reset_index(drop= True)
target_valid_0

0         10.038645
1        114.551489
2        132.603635
3        169.072125
4        122.325180
            ...    
24995    170.116726
24996     93.632175
24997    127.352259
24998     99.782700
24999    177.821022
Name: product, Length: 25000, dtype: float64

Данные подготовлены к обучению модели. В качестве модели у нас сегодня выступает только линейная регрессия. 

## Получение предсказаний на модели линейной регрессии

In [30]:
model = LinearRegression() # инициализируем модель LinearRegression

In [31]:
results = []
predictions = []



In [33]:

model_0 = LinearRegression() # инициализируем модель LinearRegression
model_0.fit(features_train_0, target_train_0) # обучим модель на тренировочной выборке
predictions_valid_0 = model_0.predict(features_valid_0) # получим предсказания модели на валидационной выборке

result_0 = mean_squared_error(target_valid_0, predictions_valid_0)**0.5 # посчитаем значение метрики RMSE на валидационной выборке
print("RMSE модели линейной регрессии на валидационной выборке 0:", result_0)

RMSE модели линейной регрессии на валидационной выборке 0: 37.5794217150813


In [34]:
#confusion_matrix(target_valid_0, predictions_valid_0)
#probabilities_valid_0 = model_0.predict_proba(features_valid_0)
#probabilities_one_valid_0 = probabilities_valid_0[:, 1]

В задании нужно сохранить правильные ответы. Но для линейной регрессии я не нашла оценку правильности ответов.Только RMSE как критерий. Наши правильные ответы - это target_valid. Так и оставляю

In [35]:
model_1 = LinearRegression() # инициализируем модель LinearRegression
model_1.fit(features_train_1, target_train_1) # обучим модель на тренировочной выборке
predictions_valid_1 = model_1.predict(features_valid_1) # получим предсказания модели на валидационной выборке
result_1 = mean_squared_error(target_valid_1, predictions_valid_1)**0.5 # посчитаем значение метрики RMSE на валидационной выборке
print("RMSE модели линейной регрессии на валидационной выборке 1:", result_1)

RMSE модели линейной регрессии на валидационной выборке 1: 0.893099286775617


In [36]:
model_2 = LinearRegression() # инициализируем модель LinearRegression
model_2.fit(features_train_2, target_train_2) # обучим модель на тренировочной выборке
predictions_valid_2 = model_2.predict(features_valid_2) # получим предсказания модели на валидационной выборке
result_2 = mean_squared_error(target_valid_2, predictions_valid_2)**0.5 # посчитаем значение метрики RMSE на валидационной выборке
print("RMSE модели линейной регрессии на валидационной выборке 2:", result_2)

RMSE модели линейной регрессии на валидационной выборке 2: 40.02970873393434


In [37]:
predictions_valid_0 = pd.Series(predictions_valid_0)
predictions_valid_1 = pd.Series(predictions_valid_1)
predictions_valid_2 = pd.Series(predictions_valid_2)
predictions = [predictions_valid_0, predictions_valid_1, predictions_valid_2]
#predictions_valid_0.reset_index()
#predictions_valid_1.reset_index()
#predictions_valid_2.reset_index()

In [38]:
results = [result_0, result_1, result_2]

У наших признаков разный масштаб и нужно стандартизировать их. Cтолбцы: f0, f1, f2, в которых данные имеют несопоставимый разброс. Чтобы избежать этой ловушки, при которой одни признаки будут ставиться выше других, приведём их к одному масштабу. Применим стандартизацию данных.

In [39]:
scaler = StandardScaler()
numeric = ['f0','f1', 'f2']
scaler.fit(features_train_0[numeric])
features_train_0_sc = features_train_0
features_valid_0_sc = features_valid_0
features_train_0_sc[numeric] = scaler.transform(features_train_0[numeric])
features_valid_0_sc[numeric] = scaler.transform(features_valid_0[numeric])

print(features_train_0_sc.head())

             f0        f1        f2
27212 -0.544828  1.390264 -0.094959
7866   1.455912 -0.480422  1.209567
62041  0.260460  0.825069 -0.204865
70185 -1.837105  0.010321 -0.147634
82230 -1.299243  0.987558  1.273181


In [40]:
scaler.fit(features_train_1[numeric])
features_train_1_sc = features_train_1
features_valid_1_sc = features_valid_1
features_train_1_sc[numeric] = scaler.transform(features_train_1[numeric])
features_valid_1_sc[numeric] = scaler.transform(features_valid_1[numeric])

print(features_train_1_sc.head())

             f0        f1        f2
27212 -0.850855  0.624428  0.296943
7866   1.971935  1.832275  0.294333
62041  1.079305  0.170127 -0.296418
70185 -1.512028 -0.887837 -0.880471
82230 -1.804775 -0.718311 -0.293255


In [41]:
scaler.fit(features_train_2[numeric])
features_train_2_sc = features_train_2
features_valid_2_sc = features_valid_2
features_train_2_sc[numeric] = scaler.transform(features_train_2[numeric])
features_valid_2_sc[numeric] = scaler.transform(features_valid_2[numeric])

print(features_train_2_sc.head())

             f0        f1        f2
27212 -0.526160  0.776329 -0.400793
7866  -0.889625 -0.404070 -1.222936
62041 -1.133984  0.208576  0.296765
70185  1.227045  1.570166 -0.764556
82230 -0.194289  0.878312  0.840821


Посмотрим, улучшились ли результаты после масштабирования. Регрессия бывает к этому очень чувствительна:

In [42]:
model_0_sc = LinearRegression() # инициализируем модель LinearRegression
model_0_sc.fit(features_train_0_sc, target_train_0) # обучим модель на тренировочной выборке
predictions_valid_0_sc = model_0_sc.predict(features_valid_0_sc) # получим предсказания модели на валидационной выборке
results_1_sc = []
result_0_sc = mean_squared_error(target_valid_0, predictions_valid_0_sc)**0.5 # посчитаем значение метрики RMSE на валидационной выборке
print("RMSE модели линейной регрессии на валидационной выборке 0:", result_0_sc)
print("Cредний запас предсказанного сырья на валидационной выборке 0:",predictions_valid_0_sc.mean() )

RMSE модели линейной регрессии на валидационной выборке 0: 37.5794217150813
Cредний запас предсказанного сырья на валидационной выборке 0: 92.59256778438035


In [43]:
model_1_sc = LinearRegression() # инициализируем модель LinearRegression
model_1_sc.fit(features_train_1_sc, target_train_1) # обучим модель на тренировочной выборке
predictions_valid_1_sc = model_1_sc.predict(features_valid_1_sc) # получим предсказания модели на валидационной выборке

result_1 = mean_squared_error(target_valid_1, predictions_valid_1_sc)**0.5 # посчитаем значение метрики RMSE на валидационной выборке
print("RMSE модели линейной регрессии на валидационной выборке 1:", result_1)
print("Cредний запас предсказанного сырья на валидационной выборке 1:",predictions_valid_1_sc.mean() )

RMSE модели линейной регрессии на валидационной выборке 1: 0.893099286775617
Cредний запас предсказанного сырья на валидационной выборке 1: 68.728546895446


In [44]:
model_2_sc = LinearRegression() # инициализируем модель LinearRegression
model_2_sc.fit(features_train_2_sc, target_train_2) # обучим модель на тренировочной выборке
predictions_valid_2_sc = model_2_sc.predict(features_valid_2) # получим предсказания модели на валидационной выборке

result_2 = mean_squared_error(target_valid_2, predictions_valid_2_sc)**0.5 # посчитаем значение метрики RMSE на валидационной выборке
print("RMSE модели линейной регрессии на валидационной выборке 2:", result_2)
print("Cредний запас предсказанного сырья на валидационной выборке 2:",predictions_valid_2_sc.mean() )

RMSE модели линейной регрессии на валидационной выборке 2: 40.02970873393434
Cредний запас предсказанного сырья на валидационной выборке 2: 94.96504596800489


И мы получили бесценный опыт, что линейная регрессия нечувствительна к масштабированию, только логистическая. Для меня удивительно.

## Подготовка к расчёту прибыли

Для расчёта прибыли нам нужно определить достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.

Для определения стоимости разработки нужно 10 млрд руб. разделить на 200 скважин. Это и будут затраты на скважину, которые определяют порог безубыточности.

In [45]:

number_of_wells = 200
well_dev_cost = COST / number_of_wells
well_dev_cost

50000000.0

50 миллионов на скважину. А сколько это в баррелях по текущей цене? По условиям проекта один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

Тут, конечно, недостаток данных в задании. Для ROI нам нужна прибыль, а не доход, а то, может, там НДПИ (налог на добычу) половину отъест. Но у нас нет даже нормы хоть какой-нибудь прибыли. Будем считать, что при стоимости нефти выше 100$ за барель, и соответствующем курсе доллара - это именно прибыль.

In [46]:
revenue_per_1000_barrel = 450000
break_even_dev = COST / revenue_per_1000_barrel
break_even_barrel = well_dev_cost / revenue_per_1000_barrel
break_even_barrel

111.11111111111111

<div class="alert alert-info">
<p><u><b>Комментарий студента:</b></u></p> 
<p>Здесь я переделала код на цикл, для красоты и правильности.
</div>

Сравним полученный необходимый объём нефти для безубыточности со средним запасом в каждом регионе.

In [47]:
for index, prediction in enumerate(predictions):
    if prediction.mean() < break_even_barrel:
        print('Средний запас в регионе', index, ' - ', prediction.mean(), 'меньше безубыточного объема')
        print('')
    else:
        print('Средний запас в регионе', index, ' - ', prediction.mean(), 'больше безубыточного объема')
        print('')



Средний запас в регионе 0  -  92.59256778438035 меньше безубыточного объема

Средний запас в регионе 1  -  68.728546895446 меньше безубыточного объема

Средний запас в регионе 2  -  94.96504596800489 меньше безубыточного объема



Во всех регионах средний запас по региону меньше безубыточного объема. А медианный?

In [48]:
for index, prediction in enumerate(predictions):
    if prediction.median() < break_even_barrel:
        print('Медианный запас в регионе ', index, ' - ', prediction.median(), 'меньше безубыточного объема')
        print('')
    else:
        print('Медианный запас в регионе', index, ' - ', prediction.median(), 'мбольше безубыточного объема')
        print('')


Медианный запас в регионе  0  -  92.66188351790804 меньше безубыточного объема

Медианный запас в регионе  1  -  57.851585882427 меньше безубыточного объема

Медианный запас в регионе  2  -  95.03120442123549 меньше безубыточного объема



И медианный тоже. 

Чило скважин, удовлеворяющмх условию безубыточности:

In [49]:

break_even_0 = predictions_valid_0[predictions_valid_0 > break_even_barrel].count()
break_even_0

5388

In [50]:
break_even_1 = predictions_valid_1[predictions_valid_1 > break_even_barrel].count()
break_even_1

4594

In [51]:
break_even_2 = predictions_valid_2[predictions_valid_2 > break_even_barrel].count()
break_even_2

5234

Их везде около 5%. Но, тем не менее, в каждой выборке есть предсказанные значения, которые выше средней стоимости разработки.

Создадим функцию для расчета прибыли

In [52]:
def profit(oil_production):
    wells_profit = oil_production.sum()*revenue_per_1000_barrel
    return wells_profit

In [53]:
profit(predictions_valid_0)

1041666387574.2789

In [54]:
profit(target_valid_0)

1035884213334.3293

## Расчёт прибыли и рисков 

Наша цель - найти те регионы, где вероятность убытков меньше 2.5% и выбрать среди них выбирают регион с наибольшей средней прибылью

Наша нулевая гипотеза H₀: «Прибыль от разработки 200 скважин в регионе ниже затрат на разработку». Альтернативная гипотеза H₁: «Прибыль от разработки 200 скважин в регионе превышает затраты на разработку».

Чтобы принять или отвергнуть нулевую гипотезу, вычислим уровень значимости — p-value. Если значение p-value больше порогового значения, то нулевую гипотезу отвергать не стоит. Меньше — есть основание отказаться от нулевой гипотезы. Общепринятое пороговое значение — 5% (для немедицинских гипотез). Нас интересует отклонение только в одну сторону — знак «больше», значит гипотеза односторонняя. Если распределение данных близко к нормальному (в данных нет существенных выбросов), для сравнения средних значений используют стандартный t-тест. 

In [187]:
alpha = 0.05  # тест односторонний: p-value будет в два раза меньше
for index, prediction in enumerate(predictions):
    results = st.ttest_ind(target_valid[index], prediction)
    pvalue = results.pvalue / 2 # < напишите код здесь >)
    print('Регион', index)
    print('p-значение: ', pvalue)
    if pvalue < alpha :
        print("Отвергаем нулевую гипотезу по региону: скорее всего скорее всего, прибыль выше затрат")
    else:
        print("Не получилось отвергнуть нулевую гипотезу по региону: скорее всего скорее всего, прибыль ниже затрат")    


Регион 0
p-значение:  0.05199864177105973
Не получилось отвергнуть нулевую гипотезу по региону: скорее всего скорее всего, прибыль ниже затрат
Регион 1
p-значение:  0.49475546644634255
Не получилось отвергнуть нулевую гипотезу по региону: скорее всего скорее всего, прибыль ниже затрат
Регион 2
p-значение:  0.397333135212901
Не получилось отвергнуть нулевую гипотезу по региону: скорее всего скорее всего, прибыль ниже затрат


Итак, без дополнительных методов выборки, средняя прибыль по всем трём регионам ниже затрат на разработку скважины.

Найдем доверительный интервал - отрезок числовой оси, в который с заданной вероятностью попадает нужный нам параметр генеральной совокупности. Нас интересует безубыточный объем выработки break_even_barrel, но считать мы будем среднее. Для вычисления доверительного интервала с каждой из его сторон обычно выбрасываем одинаковую долю экстремальных значений. В нашем случае, для построения 95%-го доверительного интервала нужно выкинуть 5% экстремальных значений, то есть 2.5% самых больших и 2.5% самых маленьких (0.025-квантиль и 0.975-квантиль)

Наши значения среднего по региону 0 лежат в скромных границах 92.31 - 92.88 с 95% вероятностью.

In [188]:
confidence_intervals = []
for index, prediction in enumerate(predictions):
    print("Cреднее по региону", index, ':', prediction.mean())
    confidence_interval  = st.t.interval(
    0.95, len(prediction)-1, prediction.mean(), prediction.sem() )
    confidence_intervals.append(confidence_interval)
    print("95%-ый доверительный интервал региона", index, ':', confidence_interval)

Cреднее по региону 0 : 92.59256778438035
95%-ый доверительный интервал региона 0 : (92.3052541593254, 92.8798814094353)
Cреднее по региону 1 : 68.728546895446
95%-ый доверительный интервал региона 1 : (68.15818110461278, 69.29891268627922)
Cреднее по региону 2 : 94.96504596800489
95%-ый доверительный интервал региона 2 : (94.71892914790764, 95.21116278810213)


Наши значения среднего по региону с 95% вероятностью лежат в скромных границах 92.59 - 92.88 по региону 0, значения среднего по региону 1 - в границах 68.16 - 69.30, по региону 2 - 94.97 - 95.21

Посмотрим свод для наглядности:

In [189]:
summary = {'Среднее':[predictions_valid_0.mean(), predictions_valid_1.mean(), predictions_valid_2.mean()], 
      'RMSE':[result_0, result_1, result_2],
      'Дов.итервал .95':confidence_intervals,
          'Прибыльных скважин':[break_even_0, break_even_1, break_even_2]}
# Таблица для удобства сравнения. 
display(pd.DataFrame(summary , index =['Регион 0', 'Регион 1', 'Регион 2' ]))

,Среднее,RMSE,Дов.итервал .95,Прибыльных скважин
Регион 0,92.592568,37.579422,"(92.3052541593254, 92.8798814094353)",5388
Регион 1,68.728547,0.893099,"(68.15818110461278, 69.29891268627922)",4594
Регион 2,94.965046,40.029709,"(94.71892914790764, 95.21116278810213)",5234


У нас есть регионы 0 и 2, где выбросов больше, но и среднее выше. И регион 1, где разброс данных меньше, но и среднее ниже.

Применим технику Bootstrap 

In [190]:
#def revenue(target, predictions, count):
#    predictions_sorted = predictions.sort_values(ascending=False)
#    selected = target[predictions_sorted.index][:count]
#    return revenue_per_1000_barrel * selected.sum()


In [55]:
def profit(target, predict):
    predict_sorted = predict.sort_values(ascending=False)
    selected_points = target[predict_sorted.index][:COUNT]
    product = selected_points.sum()
    revenue = product * PRICE * 1000
    return revenue - COST

In [ ]:
display(predictions)

In [57]:
state = np.random.RandomState(RANDOM_ST)

In [193]:
state = np.random.RandomState(RANDOM_ST)

# сохраним значения 95%-квантилей в переменных values
values = []
for index, prediction in enumerate(predictions):
    quantiles = []
    for i in range(1000):
        subsample = predictions[index].sample(n=500, replace=True, random_state=state)
        quantiles.append(pd.Series(subsample.quantile(0.95)) )
        quantiles = pd.Series(quantiles)
    values.append(quantiles)
    lower = values[index].quantile(0.0125)
    upper = values[index].quantile(0.9875)
    print(lower, 'регион', index)
    print(upper, 'регион', index)




0    128.836113
dtype: float64 регион 0
0    128.836113
dtype: float64 регион 0
0    137.230211
dtype: float64 регион 1
0    137.230211
dtype: float64 регион 1
0    127.650763
dtype: float64 регион 2
0    127.650763
dtype: float64 регион 2


In [194]:
values_0 = []
for i in range(1000):
    subsample_0 = predictions_valid_0.sample(n=500, replace=True, random_state=state)
    values_0.append(subsample_0.quantile(0.95)) 

values_0 = pd.Series(values_0) 
    
lower = values_0.quantile(0.0125)
upper = values_0.quantile(0.9875)

print(lower)
print(upper)

125.90885801442823
135.2197425842101


In [195]:
values_1 = []
for i in range(1000):
    subsample_1 = predictions_valid_1.sample(n=500, replace=True, random_state=state)
    values_1.append(subsample_1.quantile(0.95)) 

values_1 = pd.Series(values_1) 
    
lower = values_1.quantile(0.0125)
upper = values_1.quantile(0.9875)

print(lower)
print(upper)

136.51654793633472
137.69424183185353


In [196]:
values_2 = []
for i in range(1000):
    subsample_2 = predictions_valid_2.sample(n=500, replace=True, random_state=state)
    values_2.append(subsample_2.quantile(0.95)) # < напишите код здесь >

values_2 = pd.Series(values_2)
    
lower = values_2.quantile(0.0125)
upper = values_2.quantile(0.9875)

print(lower)
print(upper)

123.78568864332078
131.32302889515415


У нас получилось, что нижняя граница у всех регионов выше уровня безубыточности в 111 килобаррелей. 

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выберем регион с наибольшей средней прибылью

In [58]:
    
values_0 = []
for i in range(1000):#00):
    target_subsample_0 = target_valid_0.sample(n=500 , replace=True, random_state=state)
    probs_subsample_0 = predictions_valid_0[target_subsample_0.index]
    res_0 = profit(target_subsample_0, probs_subsample_0)# бурим только 200 скважин
    values_0.append(res_0)


In [59]:
values_0 = pd.Series(values_0)
lower = values_0.quantile(0.025)

mean = values_0.mean()
print("Средняя выручка:", mean)
print("2.5%-квантиль:", lower)

Средняя выручка: 425938526.91059244
2.5%-квантиль: -102090094.83793654


Средняя выручка: 418 763 365 рублей, но 2.5%-квантиль: -185804378, а это значит, что условие о вероятности 97,5% дохода (положительной прибыли) от бурения не выполняется. Отрицательная прибыль - это доход от продукта ниже затрат на разработку.

In [60]:
    
values_1 = []
for i in range(1000):#00):
    target_subsample_1 = target_valid_1.sample(n=500 , replace=True, random_state=state)
    probs_subsample_1 = predictions_valid_1[target_subsample_1.index]
    res_1 = profit(target_subsample_1, probs_subsample_1)# бурим  200 скважин
    values_1.append(res_1)
values_1 = pd.Series(values_1)
lower_1 = values_1.quantile(0.025)

mean_1 = values_1.mean()
print("Средняя выручка:", mean_1)
print("2.5%-квантиль:", lower_1)


Средняя выручка: 518259493.6973249
2.5%-квантиль: 128123231.43308444


In [61]:
values_2 = []
for i in range(1000):#00):
    target_subsample_2 = target_valid_2.sample(n=500 , replace=True, random_state=state)
    probs_subsample_2 = predictions_valid_2[target_subsample_2.index]
    res_2 = profit(target_subsample_2, probs_subsample_2)# бурим  200 скважин
    values_2.append(res_2)
values_2 = pd.Series(values_2)
lower_2 = values_2.quantile(0.025)

mean_2 = values_2.mean()
print("Средняя выручка:", mean_2)
print("2.5%-квантиль:", lower_2)


Средняя выручка: 420194005.3440501
2.5%-квантиль: -115852609.16001147


Регион 0 мы отбрасываем, поскольку не выполняется условие о 97.5% получения прибыли. Регионы 1 и 2 выполняют условие о прибыли. Между регионами 1 и 2 наибольшую среднюю прибыль показал регион 1, его и выбираем для разработки.

**Вывод:** по результатам проведенного анализа данных с применением модели линейная регрессия для предсказания объемов выработки скважины и техники бутстреп для оценки доверительных интервалов моделей мы предлагаем  регион 1 для разработки скважин. Число прибыльных скважин в нём 4594, что меньше, чем по другим регионам. Среднее значение продукта по исходному массиву также значительно ниже, чем по регионам 0 и 1. Однако, регион 0 мы исключаем, поскольку не выполняется условие о 97.5% получения прибыли - 2.5%-квантиль: -185804378. Регионы 1 и 2 выполняют условие о прибыли. Между регионами 1 и 2 наибольшую среднюю прибыль показал регион 1 - 531 059 970. Именно в регионе 1 мы наблюдали высокую корреляцию признака f2 и целевого признака.
Итак, после оценки рисков регионы, в которых вероятность убытков меньше 2.5% - это регионы 1 и 2 . Среди них регион с наибольшей средней прибылью - это регион 1, его мы и рекомендуем к разработке. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован